In [1]:
import warnings
from graph_generation.generate_graph import (
    generate_erdos_renyi_graph, 
    generate_erdos_renyi_attribute_graph, 
    generate_barabasi_albert_graph, 
    generate_barabasi_albert_attribute_graph,
    generate_forest_fire_graph
)
from diffusion_models.edge_probabilities import generate_edge_probabilities
from diffusion_models.independent_cascade import (
    get_best_parameters,
    optimized_independent_cascade,
    Edge_Simulation,
    Naive,
    dmp_python,
    dmp_um_fast,
    dmp_ic_fast,
    dmp_inf,
    ALE_heuristic,
    modified_ALE, 
    SWE,
    um_IC,
    IC_approx_postprocess,
    fixed_point_probs
)
from training.train_edge_gnn import train_diffusion_gnn, train_edge_model_with_diffusion, train_edge_model
import numpy as np
import networkx as nx
import torch
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data
from models.gnn_model import (
    LearnableDiffusionGNN, 
    MultiplicativeDiffusionGNN, 
    ALE, 
    ModifiedALE, 
    ICApproxLayer, 
    InfluenceSpreadNN, 
    ICApproxLossModule
)
from plotting_tools.prob_plots import posterior_plots, edge_plots
import torch.nn.functional as F
from scipy.stats import spearmanr
from sklearn.metrics import roc_curve, roc_auc_score
from scipy import stats
import time
import random
import pandas as pd
size_x=10
size_y=3.5
seed = 42
'''
random.seed(seed)                  
np.random.seed(seed)              
torch.manual_seed(seed)          
torch.cuda.manual_seed(seed)     
torch.cuda.manual_seed_all(seed)  
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
'''

/home/matic/Documents/IC_APPROX/diffusion_models/independent_cascade.py:300: SyntaxWarning: invalid escape sequence '\ '
  '''
/home/matic/Documents/IC_APPROX/diffusion_models/independent_cascade.py:727: SyntaxWarning: invalid escape sequence '\p'
  '''


'\nrandom.seed(seed)                  \nnp.random.seed(seed)              \ntorch.manual_seed(seed)          \ntorch.cuda.manual_seed(seed)     \ntorch.cuda.manual_seed_all(seed)  \ntorch.backends.cudnn.deterministic = True\ntorch.backends.cudnn.benchmark = False\n'

In [2]:
def summarise_stats(rmse, pearson, spearman, auc, time_ic_apporx, time_mc):
    ci_low, ci_high = stats.t.interval(0.95, df=len(rmse)-1, loc=rmse.mean(), scale=stats.sem(rmse))
    print(f"rmse:\n mean={rmse.mean()},\n ci_low={ci_low},\n ci_high={ci_high},\n std={rmse.std()}")

    ci_low, ci_high = stats.t.interval(0.95, df=len(pearson)-1, loc=pearson.mean(), scale=stats.sem(pearson))
    print(f"pearson:\n mean={pearson.mean()},\n ci_low={ci_low},\n ci_high={ci_high},\n std={pearson.std()}")

    ci_low, ci_high = stats.t.interval(0.95, df=len(spearman)-1, loc=spearman.mean(), scale=stats.sem(spearman))
    print(f"spearman:\n mean={spearman.mean()},\n ci_low={ci_low},\n ci_high={ci_high},\n std={spearman.std()}")

    ci_low, ci_high = stats.t.interval(0.95, df=len(auc)-1, loc=auc.mean(), scale=stats.sem(auc))
    print(f"auc:\n mean={auc.mean()},\n ci_low={ci_low},\n ci_high={ci_high},\n std={auc.std()}")

    ci_low, ci_high = stats.t.interval(0.95, df=len(time_ic_apporx)-1, loc=time_ic_apporx.mean(), scale=stats.sem(time_ic_apporx))
    print(f"time_ic_apporx:\n mean={time_ic_apporx.mean()},\n ci_low={ci_low},\n ci_high={ci_high},\n std={time_ic_apporx.std()}")

    ci_low, ci_high = stats.t.interval(0.95, df=len(time_mc)-1, loc=time_mc.mean(), scale=stats.sem(time_mc))
    print(f"time_mc:\n mean={time_mc.mean()},\n ci_low={ci_low},\n ci_high={ci_high},\n std={time_mc.std()}")

In [26]:
def append_to_dataset_csv(
    file_path,
    num_nodes,
    num_edges_per_node,
    prob_selected,
    edge_method,
    time_mc,
    approximation,
    T,
    L,
    time_ic_apporx,
    rmse,
    pearson,
    spearman,
    auc
):
    # Create a one-row DataFrame
    new_data = pd.DataFrame([{
        "num_nodes": num_nodes,
        "num_edges_per_node": num_edges_per_node,
        "prob_selected": prob_selected,
        "edge_method": edge_method,
        "time_mc": time_mc,
        "approximation": approximation,
        "T":T,
        "L":L,
        "time_ic_apporx": time_ic_apporx,
        "rmse": rmse,
        "pearson": pearson,
        "spearman": spearman,
        "auc": auc
    }])

    # Append without writing header again
    new_data.to_csv(file_path, mode='a', header=False, index=False)
    print(f"Appended new row to {file_path}")

def append_to_dataset_csv_er(
    file_path,
    num_nodes,
    p,
    prob_selected,
    edge_method,
    T,
    L,
    time_mc,
    approximation,
    time_ic_apporx,
    rmse,
    pearson,
    spearman,
    auc
):
    # Create a one-row DataFrame
    new_data = pd.DataFrame([{
        "num_nodes": num_nodes,
        "p": p,
        "prob_selected": prob_selected,
        "edge_method": edge_method,
        "time_mc": time_mc,
        "approximation": approximation,
        "T": T,
        "L": L,
        "time_ic_apporx": time_ic_apporx,
        "rmse": rmse,
        "pearson": pearson,
        "spearman": spearman,
        "auc": auc
    }])

    # Append without writing header again
    new_data.to_csv(file_path, mode='a', header=False, index=False)
    print(f"Appended new row to {file_path}")

def append_to_dataset_csv_ff(
    file_path,
    num_nodes,
    p,
    r,
    prob_selected,
    edge_method,
    time_mc,
    approximation,
    T,
    L,
    time_ic_apporx,
    rmse,
    pearson,
    spearman,
    auc
):
    # Create a one-row DataFrame
    new_data = pd.DataFrame([{
        "num_nodes": num_nodes,
        "p": p,
        "r":r,
        "prob_selected": prob_selected,
        "edge_method": edge_method,
        "time_mc": time_mc,
        "approximation": approximation,
        "T": T,
        "L": L,
        "time_ic_apporx": time_ic_apporx,
        "rmse": rmse,
        "pearson": pearson,
        "spearman": spearman,
        "auc": auc
    }])

    # Append without writing header again
    new_data.to_csv(file_path, mode='a', header=False, index=False)
    print(f"Appended new row to {file_path}")

def append_to_dataset(
    file_path,
    dataset,
    prob_selected,
    edge_method,
    time_mc,
    approximation,
    T,
    L,
    time_ic_apporx,
    rmse,
    pearson,
    spearman,
    auc
):
    # Create a one-row DataFrame
    new_data = pd.DataFrame([{
        "dataset": dataset,
        "prob_selected": prob_selected,
        "edge_method": edge_method,
        "time_mc": time_mc,
        "approximation": approximation,
        "T": T,
        "L": L,
        "time_ic_apporx": time_ic_apporx,
        "rmse": rmse,
        "pearson": pearson,
        "spearman": spearman,
        "auc": auc
    }])

    # Append without writing header again
    new_data.to_csv(file_path, mode='a', header=False, index=False)
    print(f"Appended new row to {file_path}")

## Fores Fire Graphs

### For Debugging

In [20]:
num_nodes=1000
p=0.1
r=0.1
prob_selected = 0.01
method="random"
num_graphs=1
num_sim=10000
low=0
high=0.1
percentile=95
T=5 # 5, 10, 15

G, prior_probs = generate_forest_fire_graph(n=num_nodes, p=p, r=r, prob_selected = prob_selected)
edge_dict = generate_edge_probabilities(G,method=method,low=low, high=high)

start = time.time()
ic = optimized_independent_cascade(G, prior_probs, edge_dict, num_sim)
end = time.time()
time_mc = end - start
true_posterior_tensor = torch.tensor(
    [ic[node] for node in ic],
    dtype=torch.float
).view(-1, 1)
true_probs = true_posterior_tensor.squeeze().numpy()
'''
ic2 = Edge_Simulation(G, prior_probs, edge_dict, num_sim)
true_posterior_tensor2 = torch.tensor(
    [ic2[node] for node in ic2],
    dtype=torch.float
).view(-1, 1)
true_probs2 = true_posterior_tensor2.squeeze().numpy()
'''
print("Time for MC: ", time_mc, " seconds")

Time for MC:  0.3816244602203369  seconds


In [21]:
methods = [
    "SWE",          #0
    "um_IC",        #1
    "dmp_um_fast",  #2
    "Naive",        #3
    "dmp_ic_fast",  #4
    "dmp_inf",      #5
    "modified_ALE", #6
    "ALE_heuristic" #7
]
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for method in methods:
        print(f"-------------{method}-------------")
        if method == "SWE":
            best_T, best_l = get_best_parameters(
            G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=1
            )
            start = time.time()
            preds = SWE(G, edge_dict, np.array(prior_probs), best_T, a=1, layers=best_l)
        elif method == "um_IC":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "um_IC", max_t=10)
            start = time.time()
            preds = um_IC(G, edge_dict, prior_probs, best_T)[best_T]
        elif method == "dmp_um_fast":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "dmp_um_fast")
            start = time.time()
            preds = dmp_um_fast(G, prior_probs, edge_dict, best_T, eps=1e-20)
        elif method == "Naive":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "Naive", max_t=10)
            start = time.time()
            preds = Naive(G, edge_dict, prior_probs, best_T, eps=1e-12)
        elif method == "dmp_ic_fast":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "dmp_ic_fast")
            start = time.time()
            preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)
        elif method == "dmp_inf":
            start = time.time()
            preds = dmp_inf(G, prior_probs, edge_dict, eps=1e-20, max_iter=100)
        elif method == "modified_ALE":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "modified_ALE")
            start = time.time()
            preds = modified_ALE(G, prior_probs, edge_dict, best_T)
        elif method == "ALE_heuristic":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "ALE_heuristic")
            start = time.time()
            preds = ALE_heuristic(G, prior_probs, edge_dict, best_T)
        end = time.time()
        time_ic_approx = end - start
        
        preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
        rmse = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5

        mae = abs(true_posterior_tensor.squeeze()-preds_IC_approx.squeeze()).mean()
        
        pearson = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
        spearman, _ = spearmanr(preds_IC_approx.squeeze(),
                                  true_posterior_tensor.squeeze())
        
        y_true = true_posterior_tensor.squeeze().numpy()
        y_score = preds_IC_approx.squeeze().numpy()
        threshold = np.percentile(y_true, percentile)
        y_binary = (y_true >= threshold).astype(int)
        auc = roc_auc_score(y_binary, y_score)

        if method == "SWE":
            print("best_t, best_l:        \n", best_T, best_l)
        elif method != "dmp_inf":
            print("best_t:        \n", best_T)
        print("RMSE:        \n", rmse)
        print("MAE:         \n", mae)
        print("Pearson:     \n", pearson.squeeze().numpy())
        print("Spearman:    \n", spearman)
        print("AUC:         \n", auc)
        print("Time Approx: \n", time_ic_approx)

-------------SWE-------------
best_t, best_l:        
 5 0
RMSE:        
 0.00029949704761697777
MAE:         
 tensor(5.6253e-05)
Pearson:     
 0.9999944
Spearman:    
 0.615595523423686
AUC:         
 0.9999578947368422
Time Approx: 
 0.0038673877716064453
-------------um_IC-------------
best_t:        
 3
RMSE:        
 0.00030410925291412264
MAE:         
 tensor(5.6105e-05)
Pearson:     
 0.9999949
Spearman:    
 0.8162687761141298
AUC:         
 0.999957894736842
Time Approx: 
 0.0022466182708740234
-------------dmp_um_fast-------------
best_t:        
 10
RMSE:        
 0.000310657681722372
MAE:         
 tensor(5.8172e-05)
Pearson:     
 0.99999475
Spearman:    
 0.5644491035835966
AUC:         
 0.9999578947368422
Time Approx: 
 0.012161970138549805
-------------Naive-------------
best_t:        
 3
RMSE:        
 0.00030411597019675967
MAE:         
 tensor(5.6103e-05)
Pearson:     
 0.9999948
Spearman:    
 0.8162687761141298
AUC:         
 0.999957894736842
Time Approx: 
 

### Extensive Experiments

In [21]:
num_sim=10000
percentile=95

num_graphs = 10

#p_values = [0.01, 0.03, 0.05, 0.08, 0.12]   # low→moderate→dense
#r_values = [0.0, 0.05, 0.1]                 # low→moderate clustering
p_values = [0.02, 0.05, 0.10]
r_values = [0.0, 0.05, 0.1]

pr_combine = [(p_values[i], r_values[i]) for i in range(len(p_values))] 

methods = [
    "SWE",          #0
    "um_IC",        #1
    "dmp_um_fast",  #2
    "Naive",        #3
    "dmp_ic_fast",  #4
    "dmp_inf",      #5
    "modified_ALE", #6
    "ALE_heuristic" #7
]

best_par = {}
for method in methods:
    best_par[method] = (0,0)

for num_nodes in [5000]:#[100, 1000, 10000]:
    #for p in p_values:
        #for r in r_values:
    for p, r in pr_combine:
        for prob_selected in [0.01, 0.1, 0.2]:
            #for high in [0.1, 0.5, 1]:
            for i in range(num_graphs+1):
                G, prior_probs = generate_forest_fire_graph(n=num_nodes, 
                                                            p=p, 
                                                            r=r, 
                                                            prob_selected = prob_selected)
                
                edge_dict = generate_edge_probabilities(G,method="degree_based",low=0, high=high)
            
                start = time.time()
                ic = optimized_independent_cascade(G, prior_probs, edge_dict, num_sim)
                end = time.time()
                time_mc = end - start
                true_posterior_tensor = torch.tensor(
                    [ic[node] for node in ic],
                    dtype=torch.float
                ).view(-1, 1)
                true_probs = true_posterior_tensor.squeeze().numpy()

                for approximation in methods:
                    if approximation == "SWE":
                        if i==0:
                            best_T, best_l = get_best_parameters(
                            G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
                            )
                            best_par[approximation] = (best_T, best_l)
                            continue
                        best_T, best_l = best_par[approximation]
                        start = time.time()
                        preds = SWE(G, edge_dict, prior_probs, T=best_T, a=1, layers=best_l)
                        end = time.time()
                        time_ic_apporx = end - start
                    elif approximation == "um_IC":
                        if i==0:
                            best_T = get_best_parameters(
                            G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                            )
                            best_par[approximation] = (best_T, 0)
                            continue
                        best_T, best_l = best_par[approximation]
                        start = time.time()
                        preds = um_IC(G, edge_dict, prior_probs, best_T)[best_T]
                        end = time.time()
                        time_ic_apporx = end - start
                    elif approximation == "dmp_um_fast":
                        if i==0:
                            best_T = get_best_parameters(
                            G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                            )
                            best_par[approximation] = (best_T, 0)
                            continue
                        best_T, best_l = best_par[approximation]
                        start = time.time()
                        preds = dmp_um_fast(G, prior_probs, edge_dict, best_T, eps=1e-20)
                        end = time.time()
                        time_ic_apporx = end - start
                    elif approximation == "Naive":
                        if i==0:
                            best_T = get_best_parameters(
                            G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                            )
                            best_par[approximation] = (best_T, 0)
                            continue
                        best_T, best_l = best_par[approximation]
                        start = time.time()
                        preds = Naive(G, edge_dict, prior_probs, best_T, eps=1e-12)
                        end = time.time()
                        time_ic_apporx = end - start
                    elif approximation == "dmp_ic_fast":
                        if i==0:
                            best_T = get_best_parameters(
                            G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                            )
                            best_par[approximation] = (best_T, 0)
                            continue
                        best_T, best_l = best_par[approximation]
                        start = time.time()
                        preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)
                        end = time.time()
                        time_ic_apporx = end - start
                    elif approximation == "dmp_inf":
                        if i==0:
                            best_par[approximation] = (0, 0)
                            continue
                        start = time.time()
                        preds = dmp_inf(G, prior_probs, edge_dict, eps=1e-20, max_iter=100)
                        end = time.time()
                        time_ic_apporx = end - start
                    elif approximation == "modified_ALE":
                        if i==0:
                            best_T = get_best_parameters(
                            G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                            )
                            best_par[approximation] = (best_T, 0)
                            continue
                        best_T, best_l = best_par[approximation]
                        start = time.time()
                        preds = modified_ALE(G, prior_probs, edge_dict, best_T)
                        end = time.time()
                        time_ic_apporx = end - start
                    elif approximation == "ALE_heuristic":
                        if i==0:
                            best_T = get_best_parameters(
                            G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                            )
                            best_par[approximation] = (best_T, 0)
                            continue
                        best_T, best_l = best_par[approximation]
                        start = time.time()
                        preds = ALE_heuristic(G, prior_probs, edge_dict, best_T)
                        end = time.time()
                        time_ic_apporx = end - start
                    
                    preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
                    rmse = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5
                
                    pearson = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
                    spearman, _ = spearmanr(preds_IC_approx.squeeze(),
                                              true_posterior_tensor.squeeze())
                
                    y_true = true_posterior_tensor.squeeze().numpy()
                    y_score = preds_IC_approx.squeeze().numpy()
                    threshold = np.percentile(y_true, percentile)
                    y_binary = (y_true >= threshold).astype(int)
                    auc = roc_auc_score(y_binary, y_score)

                    edge_method = "degree_based"#"random_" + str(high)

                    append_to_dataset_csv_ff(
                        file_path="IC_APPROX_FF.csv",
                        num_nodes=num_nodes,
                        p=p,
                        r=r,
                        prob_selected=prob_selected,
                        edge_method=edge_method,
                        time_mc=time_mc,
                        approximation=approximation,
                        time_ic_apporx=time_ic_apporx,
                        T=best_T,
                        L=best_l,
                        rmse=rmse,
                        pearson=pearson,
                        spearman=spearman,
                        auc=auc
                    )
                print("num_nodes: ", num_nodes, 
                      ", p: ", p, 
                      ", r: ", r,
                      ", prob_selected: ", prob_selected,
                      #", high: ", high, 
                      ", graph: ", i)

/tmp/ipykernel_3455152/826942034.py:56: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
/tmp/ipykernel_3455152/826942034.py:68: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
/tmp/ipykernel_3455152/826942034.py:80: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so pas

num_nodes:  5000 , p:  0.02 , r:  0.0 , prob_selected:  0.01 , graph:  0
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv


/tmp/ipykernel_3455152/826942034.py:110: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
num_nodes:  5000 , p:  0.02 , r:  0.0 , prob_selected:  0.01 , graph:  1
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv


/tmp/ipykernel_3455152/826942034.py:110: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


...

/tmp/ipykernel_3455152/826942034.py:110: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
num_nodes:  5000 , p:  0.1 , r:  0.1 , prob_selected:  0.2 , graph:  8
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv


/tmp/ipykernel_3455152/826942034.py:110: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
num_nodes:  5000 , p:  0.1 , r:  0.1 , prob_selected:  0.2 , graph:  9
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
Appended new row to IC_APPROX_FF.csv
num_nodes:  5000 , p:  0.1 , r:  0.1 , prob_selected:  0.2 , graph:  10


/tmp/ipykernel_3455152/826942034.py:110: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


## Barabasi Albert

### For Debugging

In [5]:
num_nodes=10000
num_edges_per_node=2 #2,5,10
prob_selected = 0.01
method="degree_based"
num_sim=10000
low=0
high=0.1
percentile=95
T=5 # 5, 10, 15

G, prior_probs = generate_barabasi_albert_graph(num_nodes = num_nodes,
                                                num_edges_per_node = num_edges_per_node,
                                                prob_selected = prob_selected)

edge_dict = generate_edge_probabilities(G,method=method,low=low, high=high)
for edge in edge_dict:
    edge_dict[edge] = edge_dict[edge]**3
#prior_probs = np.zeros(num_nodes)
#prior_probs[random.randint(0, num_nodes-1)]=1

start = time.time()
ic = optimized_independent_cascade(G, prior_probs, edge_dict, num_sim)
end = time.time()
time_mc = end - start
true_posterior_tensor = torch.tensor(
    [ic[node] for node in ic],
    dtype=torch.float
).view(-1, 1)
true_probs = true_posterior_tensor.squeeze().numpy()
print("Time for MC: ", time_mc, " seconds")

Time for MC:  2.2782373428344727  seconds


In [6]:
methods = [
    "SWE",          #0
    "um_IC",        #1
    "dmp_um_fast",  #2
    "Naive",        #3
    "dmp_ic_fast",  #4
    "dmp_inf",      #5
    "modified_ALE", #6
    "ALE_heuristic" #7
]
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for method in methods:
        print(f"-------------{method}-------------")
        if method == "SWE":
            best_T, best_l = get_best_parameters(
            G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
            )
            start = time.time()
            preds = SWE(G, edge_dict, np.array(prior_probs), best_T, a=1, layers=best_l)
        elif method == "um_IC":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "um_IC", max_t=10)
            start = time.time()
            preds = um_IC(G, edge_dict, prior_probs, best_T)[best_T]
        elif method == "dmp_um_fast":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "dmp_um_fast")
            start = time.time()
            preds = dmp_um_fast(G, prior_probs, edge_dict, best_T, eps=1e-20)
        elif method == "Naive":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "Naive", max_t=10)
            start = time.time()
            preds = Naive(G, edge_dict, prior_probs, best_T, eps=1e-12)
        elif method == "dmp_ic_fast":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "dmp_ic_fast")
            start = time.time()
            preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)
        elif method == "dmp_inf":
            start = time.time()
            preds = dmp_inf(G, prior_probs, edge_dict, eps=1e-20, max_iter=100)
        elif method == "modified_ALE":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "modified_ALE")
            start = time.time()
            preds = modified_ALE(G, prior_probs, edge_dict, best_T)
        elif method == "ALE_heuristic":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "ALE_heuristic")
            start = time.time()
            preds = ALE_heuristic(G, prior_probs, edge_dict, best_T)
        end = time.time()
        time_ic_approx = end - start
        
        preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
        rmse = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5

        mae = abs(true_posterior_tensor.squeeze()-preds_IC_approx.squeeze()).mean()
        
        pearson = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
        spearman, _ = spearmanr(preds_IC_approx.squeeze(),
                                  true_posterior_tensor.squeeze())
        
        y_true = true_posterior_tensor.squeeze().numpy()
        y_score = preds_IC_approx.squeeze().numpy()
        threshold = np.percentile(y_true, percentile)
        y_binary = (y_true >= threshold).astype(int)
        auc = roc_auc_score(y_binary, y_score)

        if method == "SWE":
            print("best_t, best_l:        \n", best_T, best_l)
        elif method != "dmp_inf":
            print("best_t:        \n", best_T)
        print("RMSE:        \n", rmse)
        print("MAE:         \n", mae)
        print("Pearson:     \n", pearson.squeeze().numpy())
        print("Spearman:    \n", spearman)
        print("AUC:         \n", auc)
        print("Time Approx: \n", time_ic_approx)

-------------SWE-------------
best_t, best_l:        
 1 1
RMSE:        
 0.00044533305498817126
MAE:         
 tensor(5.7966e-05)
Pearson:     
 0.9999687
Spearman:    
 0.5056757612798695
AUC:         
 0.9903173765548439
Time Approx: 
 0.029410839080810547
-------------um_IC-------------
best_t:        
 2
RMSE:        
 0.0004452173639565424
MAE:         
 tensor(5.7939e-05)
Pearson:     
 0.9999686
Spearman:    
 0.5056757612798695
AUC:         
 0.9903173765548438
Time Approx: 
 0.034276485443115234
-------------dmp_um_fast-------------
best_t:        
 8
RMSE:        
 0.07383276304178311
MAE:         
 tensor(0.0096)
Pearson:     
 0.010794087
Spearman:    
 0.006022615114389372
AUC:         
 0.5014802864681492
Time Approx: 
 0.12575531005859375
-------------Naive-------------
best_t:        
 1
RMSE:        
 0.07383255175786521
MAE:         
 tensor(0.0096)
Pearson:     
 0.010814588
Spearman:    
 -0.008407246122611156
AUC:         
 0.493564317125267
Time Approx: 
 0.07755

### Extensive Experiments

In [25]:
num_sim=10000
percentile=95
T=5 # 5, 10, 15

num_graphs = 10

for num_nodes in [5000]:#[100, 1000, 10000]:
    for num_edges_per_node in range(1,11):
        for prob_selected in [0.01, 0.1, 0.2]:
            #for high in [0.1, 0.5, 1]:
            for i in range(num_graphs+1):
                G, prior_probs = generate_barabasi_albert_graph(num_nodes = num_nodes,
                                                                num_edges_per_node = num_edges_per_node,
                                                                prob_selected = prob_selected)
                
                edge_dict = generate_edge_probabilities(G,method="degree_based",low=0, high=high)
                
                start = time.time()
                ic = optimized_independent_cascade(G, prior_probs, edge_dict, num_sim)
                end = time.time()
                time_mc = end - start
                true_posterior_tensor = torch.tensor(
                    [ic[node] for node in ic],
                    dtype=torch.float
                ).view(-1, 1)
                true_probs = true_posterior_tensor.squeeze().numpy()

                for approximation in methods:
                    if approximation == "SWE":
                        if i==0:
                            best_T, best_l = get_best_parameters(
                            G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
                            )
                            best_par[approximation] = (best_T, best_l)
                            continue
                        best_T, best_l = best_par[approximation]
                        start = time.time()
                        preds = SWE(G, edge_dict, prior_probs, T=best_T, a=1, layers=best_l)
                        end = time.time()
                        time_ic_apporx = end - start
                    elif approximation == "um_IC":
                        if i==0:
                            best_T = get_best_parameters(
                            G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                            )
                            best_par[approximation] = (best_T, 0)
                            continue
                        best_T, best_l = best_par[approximation]
                        start = time.time()
                        preds = um_IC(G, edge_dict, prior_probs, best_T)[best_T]
                        end = time.time()
                        time_ic_apporx = end - start
                    elif approximation == "dmp_um_fast":
                        if i==0:
                            best_T = get_best_parameters(
                            G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                            )
                            best_par[approximation] = (best_T, 0)
                            continue
                        best_T, best_l = best_par[approximation]
                        start = time.time()
                        preds = dmp_um_fast(G, prior_probs, edge_dict, best_T, eps=1e-20)
                        end = time.time()
                        time_ic_apporx = end - start
                    elif approximation == "Naive":
                        if i==0:
                            best_T = get_best_parameters(
                            G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                            )
                            best_par[approximation] = (best_T, 0)
                            continue
                        best_T, best_l = best_par[approximation]
                        start = time.time()
                        preds = Naive(G, edge_dict, prior_probs, best_T, eps=1e-12)
                        end = time.time()
                        time_ic_apporx = end - start
                    elif approximation == "dmp_ic_fast":
                        if i==0:
                            best_T = get_best_parameters(
                            G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                            )
                            best_par[approximation] = (best_T, 0)
                            continue
                        best_T, best_l = best_par[approximation]
                        start = time.time()
                        preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)
                        end = time.time()
                        time_ic_apporx = end - start
                    elif approximation == "dmp_inf":
                        if i==0:
                            best_par[approximation] = (0, 0)
                            continue
                        start = time.time()
                        preds = dmp_inf(G, prior_probs, edge_dict, eps=1e-20, max_iter=100)
                        end = time.time()
                        time_ic_apporx = end - start
                    elif approximation == "modified_ALE":
                        if i==0:
                            best_T = get_best_parameters(
                            G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                            )
                            best_par[approximation] = (best_T, 0)
                            continue
                        best_T, best_l = best_par[approximation]
                        start = time.time()
                        preds = modified_ALE(G, prior_probs, edge_dict, best_T)
                        end = time.time()
                        time_ic_apporx = end - start
                    elif approximation == "ALE_heuristic":
                        if i==0:
                            best_T = get_best_parameters(
                            G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                            )
                            best_par[approximation] = (best_T, 0)
                            continue
                        best_T, best_l = best_par[approximation]
                        start = time.time()
                        preds = ALE_heuristic(G, prior_probs, edge_dict, best_T)
                        end = time.time()
                        time_ic_apporx = end - start
                    
                    preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
                    rmse = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5
                
                    pearson = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
                    spearman, _ = spearmanr(preds_IC_approx.squeeze(),
                                              true_posterior_tensor.squeeze())
                
                    y_true = true_posterior_tensor.squeeze().numpy()
                    y_score = preds_IC_approx.squeeze().numpy()
                    threshold = np.percentile(y_true, percentile)
                    y_binary = (y_true >= threshold).astype(int)
                    auc = roc_auc_score(y_binary, y_score)

                    edge_method = "degree_based"#"random_" + str(high)

                    append_to_dataset_csv(
                        file_path="IC_APPROX_BA.csv",
                        num_nodes=num_nodes,
                        num_edges_per_node=num_edges_per_node,
                        prob_selected=prob_selected,
                        edge_method=edge_method,
                        time_mc=time_mc,
                        approximation=approximation,
                        T=best_T,
                        L=best_l,
                        time_ic_apporx=time_ic_apporx,
                        rmse=rmse,
                        pearson=pearson,
                        spearman=spearman,
                        auc=auc
                    )
                print("num_nodes: ", num_nodes, 
                      ", num_edges_per_node: ", num_edges_per_node, 
                      ", prob_selected: ", prob_selected,
                      #", high: ", high, 
                      ", graph: ", i)

/tmp/ipykernel_3455152/734448758.py:32: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
/tmp/ipykernel_3455152/734448758.py:44: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
/tmp/ipykernel_3455152/734448758.py:56: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so pas

num_nodes:  5000 , num_edges_per_node:  1 , prob_selected:  0.01 , graph:  0
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv


/tmp/ipykernel_3455152/734448758.py:86: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


Appended new row to IC_APPROX_BA.csv
num_nodes:  5000 , num_edges_per_node:  1 , prob_selected:  0.01 , graph:  1
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
num_nodes:  5000 , num_edges_per_node:  1 , prob_selected:  0.01 , graph:  2


/tmp/ipykernel_3455152/734448758.py:86: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


...

Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
num_nodes:  5000 , num_edges_per_node:  10 , prob_selected:  0.2 , graph:  9


/tmp/ipykernel_3455152/734448758.py:122: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)


Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv


/tmp/ipykernel_3455152/734448758.py:86: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
Appended new row to IC_APPROX_BA.csv
num_nodes:  5000 , num_edges_per_node:  10 , prob_selected:  0.2 , graph:  10


## Erdos Renyi

### For Debugging

In [38]:
num_nodes=1000
edge_prob=0.01 #0.0004, 0.0010, 0.0020
prob_selected = 0.01
method="random"
num_sim=10000
low=0
high=0.1
percentile=95
T=5 # 5, 10, 15

G, prior_probs = generate_erdos_renyi_graph(num_nodes = num_nodes,
                                                edge_prob = edge_prob,
                                                prob_selected = prob_selected,
                                                seed=seed)

edge_dict = generate_edge_probabilities(G,method=method,low=low, high=high)

start = time.time()
ic = optimized_independent_cascade(G, prior_probs, edge_dict, num_sim)
end = time.time()
time_mc = end - start
true_posterior_tensor = torch.tensor(
    [ic[node] for node in ic],
    dtype=torch.float
).view(-1, 1)
true_probs = true_posterior_tensor.squeeze().numpy()
print("Time for MC: ", time_mc, " seconds")

Time for MC:  2.352745294570923  seconds


In [39]:
methods = [
    "SWE",          #0
    "um_IC",        #1
    "dmp_um_fast",  #2
    "Naive",        #3
    "dmp_ic_fast",  #4
    "dmp_inf",      #5
    "modified_ALE", #6
    "ALE_heuristic" #7
]
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for method in methods:
        print(f"-------------{method}-------------")
        if method == "SWE":
            best_T, best_l = get_best_parameters(
            G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
            )
            start = time.time()
            preds = SWE(G, edge_dict, np.array(prior_probs), best_T, a=1, layers=best_l)
        elif method == "um_IC":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "um_IC", max_t=10)
            start = time.time()
            preds = um_IC(G, edge_dict, prior_probs, best_T)[best_T]
        elif method == "dmp_um_fast":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "dmp_um_fast")
            start = time.time()
            preds = dmp_um_fast(G, prior_probs, edge_dict, best_T, eps=1e-20)
        elif method == "Naive":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "Naive", max_t=10)
            start = time.time()
            preds = Naive(G, edge_dict, prior_probs, best_T, eps=1e-12)
        elif method == "dmp_ic_fast":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "dmp_ic_fast")
            start = time.time()
            preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)
        elif method == "dmp_inf":
            start = time.time()
            preds = dmp_inf(G, prior_probs, edge_dict, eps=1e-20, max_iter=100)
        elif method == "modified_ALE":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "modified_ALE")
            start = time.time()
            preds = modified_ALE(G, prior_probs, edge_dict, best_T)
        elif method == "ALE_heuristic":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "ALE_heuristic")
            start = time.time()
            preds = ALE_heuristic(G, prior_probs, edge_dict, best_T)
        end = time.time()
        time_ic_approx = end - start
        
        preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
        rmse = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5

        mae = abs(true_posterior_tensor.squeeze()-preds_IC_approx.squeeze()).mean()
        
        pearson = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
        spearman, _ = spearmanr(preds_IC_approx.squeeze(),
                                  true_posterior_tensor.squeeze())
        
        y_true = true_posterior_tensor.squeeze().numpy()
        y_score = preds_IC_approx.squeeze().numpy()
        threshold = np.percentile(y_true, percentile)
        y_binary = (y_true >= threshold).astype(int)
        auc = roc_auc_score(y_binary, y_score)

        if method == "SWE":
            print("best_t, best_l:        \n", best_T, best_l)
        elif method != "dmp_inf":
            print("best_t:        \n", best_T)
        print("RMSE:        \n", rmse)
        print("MAE:         \n", mae)
        print("Pearson:     \n", pearson.squeeze().numpy())
        print("Spearman:    \n", spearman)
        print("AUC:         \n", auc)
        print("Time Approx: \n", time_ic_approx)

-------------SWE-------------
best_t, best_l:        
 5 0
RMSE:        
 0.001402797055947427
MAE:         
 tensor(0.0010)
Pearson:     
 0.9999422
Spearman:    
 0.9908348576047108
AUC:         
 0.9998526315789473
Time Approx: 
 0.0069446563720703125
-------------um_IC-------------
best_t:        
 5
RMSE:        
 0.0015350319104433824
MAE:         
 tensor(0.0010)
Pearson:     
 0.99993205
Spearman:    
 0.990843347806244
AUC:         
 0.9998526315789473
Time Approx: 
 0.006058216094970703
-------------dmp_um_fast-------------
best_t:        
 6
RMSE:        
 0.001321141521621069
MAE:         
 tensor(0.0009)
Pearson:     
 0.99994814
Spearman:    
 0.9909181575820156
AUC:         
 0.9998736842105264
Time Approx: 
 0.018215417861938477
-------------Naive-------------
best_t:        
 5
RMSE:        
 0.0015376445041246235
MAE:         
 tensor(0.0010)
Pearson:     
 0.99993193
Spearman:    
 0.9908413257582462
AUC:         
 0.9998526315789473
Time Approx: 
 0.0141973495483398

### Extensive Experiments

In [34]:
num_sim=10000
percentile=95
T=5 # 5, 10, 15

num_graphs = 2

for num_nodes in [5000]:#[100, 1000, 10000]:
    for p in [2*j/num_nodes for j in range(1,11)]:
        for prob_selected in [0.01, 0.1, 0.2]:
            for high in [0.05, 0.1, 0.5, 1]:
                for i in range(num_graphs):
                    G, prior_probs = generate_erdos_renyi_graph(num_nodes = num_nodes,
                                                    edge_prob = p,
                                                    prob_selected = prob_selected,
                                                    seed=seed)
                    
                    edge_dict = generate_edge_probabilities(G,method="random",low=0, high=high)
                
                    start = time.time()
                    ic = optimized_independent_cascade(G, prior_probs, edge_dict, num_sim)
                    end = time.time()
                    time_mc = end - start
                    true_posterior_tensor = torch.tensor(
                        [ic[node] for node in ic],
                        dtype=torch.float
                    ).view(-1, 1)
                    true_probs = true_posterior_tensor.squeeze().numpy()
        
                    for approximation in methods:
                        if approximation == "SWE":
                            if i==0:
                                best_T, best_l = get_best_parameters(
                                G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
                                )
                                best_par[approximation] = (best_T, best_l)
                                continue
                            best_T, best_l = best_par[approximation]
                            start = time.time()
                            preds = SWE(G, edge_dict, prior_probs, T=best_T, a=1, layers=best_l)
                            end = time.time()
                            time_ic_apporx = end - start
                        elif approximation == "um_IC":
                            if i==0:
                                best_T = get_best_parameters(
                                G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                                )
                                best_par[approximation] = (best_T, 0)
                                continue
                            best_T, best_l = best_par[approximation]
                            start = time.time()
                            preds = um_IC(G, edge_dict, prior_probs, best_T)[best_T]
                            end = time.time()
                            time_ic_apporx = end - start
                        elif approximation == "dmp_um_fast":
                            if i==0:
                                best_T = get_best_parameters(
                                G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                                )
                                best_par[approximation] = (best_T, 0)
                                continue
                            best_T, best_l = best_par[approximation]
                            start = time.time()
                            preds = dmp_um_fast(G, prior_probs, edge_dict, best_T, eps=1e-20)
                            end = time.time()
                            time_ic_apporx = end - start
                        elif approximation == "Naive":
                            if i==0:
                                best_T = get_best_parameters(
                                G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                                )
                                best_par[approximation] = (best_T, 0)
                                continue
                            best_T, best_l = best_par[approximation]
                            start = time.time()
                            preds = Naive(G, edge_dict, prior_probs, best_T, eps=1e-12)
                            end = time.time()
                            time_ic_apporx = end - start
                        elif approximation == "dmp_ic_fast":
                            if i==0:
                                best_T = get_best_parameters(
                                G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                                )
                                best_par[approximation] = (best_T, 0)
                                continue
                            best_T, best_l = best_par[approximation]
                            start = time.time()
                            preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)
                            end = time.time()
                            time_ic_apporx = end - start
                        elif approximation == "dmp_inf":
                            if i==0:
                                best_par[approximation] = (0, 0)
                                continue
                            start = time.time()
                            preds = dmp_inf(G, prior_probs, edge_dict, eps=1e-20, max_iter=100)
                            end = time.time()
                            time_ic_apporx = end - start
                        elif approximation == "modified_ALE":
                            if i==0:
                                best_T = get_best_parameters(
                                G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                                )
                                best_par[approximation] = (best_T, 0)
                                continue
                            best_T, best_l = best_par[approximation]
                            start = time.time()
                            preds = modified_ALE(G, prior_probs, edge_dict, best_T)
                            end = time.time()
                            time_ic_apporx = end - start
                        elif approximation == "ALE_heuristic":
                            if i==0:
                                best_T = get_best_parameters(
                                G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                                )
                                best_par[approximation] = (best_T, 0)
                                continue
                            best_T, best_l = best_par[approximation]
                            start = time.time()
                            preds = ALE_heuristic(G, prior_probs, edge_dict, best_T)
                            end = time.time()
                            time_ic_apporx = end - start
                        
                        preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
                        rmse = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5
                    
                        pearson = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
                        spearman, _ = spearmanr(preds_IC_approx.squeeze(),
                                                  true_posterior_tensor.squeeze())
                    
                        y_true = true_posterior_tensor.squeeze().numpy()
                        y_score = preds_IC_approx.squeeze().numpy()
                        threshold = np.percentile(y_true, percentile)
                        y_binary = (y_true >= threshold).astype(int)
                        auc = roc_auc_score(y_binary, y_score)
        
                        edge_method = "random_" + str(high)
        
                        append_to_dataset_csv_er(
                            file_path="IC_APPROX_ER.csv",
                            num_nodes=num_nodes,
                            p=p,
                            prob_selected=prob_selected,
                            edge_method=edge_method,
                            time_mc=time_mc,
                            approximation=approximation,
                            T=best_T,
                            L=best_l,
                            time_ic_apporx=time_ic_apporx,
                            rmse=rmse,
                            pearson=pearson,
                            spearman=spearman,
                            auc=auc
                        )
                    print("num_nodes: ", num_nodes, 
                          ", p: ", p, 
                          ", prob_selected: ", prob_selected,
                          ", high: ", high, 
                          ", graph: ", i)


/tmp/ipykernel_3455152/975726324.py:33: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
/tmp/ipykernel_3455152/975726324.py:45: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
/tmp/ipykernel_3455152/975726324.py:57: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so pas

num_nodes:  5000 , p:  0.0004 , prob_selected:  0.01 , high:  0.05 , graph:  0
Appended new row to IC_APPROX_ER.csv
Appended new row to IC_APPROX_ER.csv
Appended new row to IC_APPROX_ER.csv
Appended new row to IC_APPROX_ER.csv
Appended new row to IC_APPROX_ER.csv
Appended new row to IC_APPROX_ER.csv


/tmp/ipykernel_3455152/975726324.py:87: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


Appended new row to IC_APPROX_ER.csv
Appended new row to IC_APPROX_ER.csv
num_nodes:  5000 , p:  0.0004 , prob_selected:  0.01 , high:  0.05 , graph:  1


/tmp/ipykernel_3455152/975726324.py:33: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
/tmp/ipykernel_3455152/975726324.py:45: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
/tmp/ipykernel_3455152/975726324.py:57: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so pas

...

/tmp/ipykernel_3455152/975726324.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)


Appended new row to IC_APPROX_ER.csv
Appended new row to IC_APPROX_ER.csv
Appended new row to IC_APPROX_ER.csv
Appended new row to IC_APPROX_ER.csv


/tmp/ipykernel_3455152/975726324.py:87: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


Appended new row to IC_APPROX_ER.csv
Appended new row to IC_APPROX_ER.csv
Appended new row to IC_APPROX_ER.csv
Appended new row to IC_APPROX_ER.csv
num_nodes:  5000 , p:  0.004 , prob_selected:  0.2 , high:  1 , graph:  1


/tmp/ipykernel_3455152/975726324.py:127: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman, _ = spearmanr(preds_IC_approx.squeeze(),


## Real World Graphs

In [3]:
def load_directed_graph_from_edges(file_path):
    """
    Reads a file containing edge list data and creates a directed graph using NetworkX.

    Each line in the file should contain two integers: source target

    Parameters:
    - file_path (str): Path to the edge list file.

    Returns:
    - G (networkx.DiGraph): Directed graph constructed from the edge list.
    """
    G = nx.DiGraph()  # Directed graph

    with open(file_path, 'r') as f:
        for line in f:
            if line.strip():  # skip empty lines
                source, target = map(int, line.strip().split())
                G.add_edge(source-1, target-1)

    return G

def load_weighted_directed_graph(file_path):
    """
    Loads a weighted directed graph from a file.
    Ignores lines starting with '%'. Assumes each valid line has: source target weight

    Returns:
    - G (nx.DiGraph): The directed graph.
    - edge_weight (dict): Dictionary with (u, v) -> weight mapping.
    """
    G = nx.DiGraph()
    edge_weight = {}

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('%'):
                continue  # Skip comments and empty lines

            parts = line.split()
            if len(parts) != 3:
                continue  # Skip malformed lines

            u, v = int(parts[0])-1, int(parts[1])-1
            w = float(parts[2])

            G.add_edge(u, v, weight=w)
            G.add_edge(v, u, weight=w)
            edge_weight[(u, v)] = w
            edge_weight[(v, u)] = w

    return G, edge_weight

def load_directed_graph_from_edges_cs(file_path):
    """
    Reads a file containing edge list data and creates a directed graph using NetworkX.

    Each line in the file should contain two integers: source target

    Parameters:
    - file_path (str): Path to the edge list file.

    Returns:
    - G (networkx.DiGraph): Directed graph constructed from the edge list.
    """
    G = nx.DiGraph()  # Directed graph

    with open(file_path, 'r') as f:
        for line in f:
            if line.strip():  # skip empty lines
                source, target = map(int, line.strip().split(","))
                G.add_edge(source, target)

    return G

import networkx as nx

def load_directed_graph_from_mtx(file_path):
    """
    Reads a Matrix Market (.mtx) file containing edge list data and 
    creates a directed graph using NetworkX.

    Each non-comment line after the header should contain three integers:
    source target weight

    The weight is ignored.

    Parameters:
    - file_path (str): Path to the .mtx file.

    Returns:
    - G (networkx.DiGraph): Directed graph constructed from the edge list.
    """
    G = nx.DiGraph()

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('%'):
                continue  # skip comments and blank lines
            parts = line.split()
            if len(parts) == 3:  # source target weight
                source, target = map(int, parts[:2])
                G.add_edge(source-1, target-1)

    return G

## Anybeat

### For Debugging

In [4]:
prob_selected = 0.01
method="random"
num_sim=1000
low=0
high=0.1
percentile=95
T=5

G_anybeat = load_directed_graph_from_edges("soc-anybeat/soc-anybeat.edges")
edge_dict_anybeat = generate_edge_probabilities(G_anybeat,method=method,low=low, high=high)
    
mask = torch.rand(len(G_anybeat.nodes())) < prob_selected
prior_probs_anybeat = mask.float()  * torch.rand(len(G_anybeat.nodes()))

start = time.time()
ic_anybeat = optimized_independent_cascade(G_anybeat, prior_probs_anybeat, edge_dict_anybeat, num_sim)
end = time.time()
time_mc_anybeat = end - start
true_posterior_tensor_anybeat = torch.tensor(
    [ic_anybeat[node] for node in ic_anybeat],
    dtype=torch.float
).view(-1, 1)
true_probs = true_posterior_tensor_anybeat.squeeze().numpy()
print("Time for MC: ", time_mc_anybeat, " seconds")

Time for MC:  16.587751150131226  seconds


In [13]:
G = G_anybeat
edge_dict = edge_dict_anybeat
prior_probs = prior_probs_anybeat
methods = [
    "SWE",          #0
    "um_IC",        #1
    "dmp_um_fast",  #2
    "Naive",        #3
    "dmp_ic_fast",  #4
    "dmp_inf",      #5
    "modified_ALE", #6
    "ALE_heuristic" #7
]
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for method in methods:
        print(f"-------------{method}-------------")
        if method == "SWE":
            best_T, best_l = get_best_parameters(
            G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
            )
            start = time.time()
            preds = SWE(G, edge_dict, np.array(prior_probs), best_T, a=1, layers=best_l)
        elif method == "um_IC":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "um_IC", max_t=10)
            start = time.time()
            preds = um_IC(G, edge_dict, prior_probs, best_T)[best_T]
        elif method == "dmp_um_fast":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "dmp_um_fast")
            start = time.time()
            preds = dmp_um_fast(G, prior_probs, edge_dict, best_T, eps=1e-20)
        elif method == "Naive":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "Naive", max_t=10)
            start = time.time()
            preds = Naive(G, edge_dict, prior_probs, best_T, eps=1e-12)
        elif method == "dmp_ic_fast":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "dmp_ic_fast")
            start = time.time()
            preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)
        elif method == "dmp_inf":
            start = time.time()
            preds = dmp_inf(G, prior_probs, edge_dict, eps=1e-20, max_iter=100)
        elif method == "modified_ALE":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "modified_ALE")
            start = time.time()
            preds = modified_ALE(G, prior_probs, edge_dict, best_T)
        elif method == "ALE_heuristic":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "ALE_heuristic")
            start = time.time()
            preds = ALE_heuristic(G, prior_probs, edge_dict, best_T)
        end = time.time()
        time_ic_approx = end - start

        true_posterior_tensor = true_posterior_tensor_anybeat
        preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
        rmse = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5

        mae = abs(true_posterior_tensor.squeeze()-preds_IC_approx.squeeze()).mean()
        
        pearson = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
        spearman, _ = spearmanr(preds_IC_approx.squeeze(),
                                  true_posterior_tensor.squeeze())
        
        y_true = true_posterior_tensor.squeeze().numpy()
        y_score = preds_IC_approx.squeeze().numpy()
        threshold = np.percentile(y_true, percentile)
        y_binary = (y_true >= threshold).astype(int)
        auc = roc_auc_score(y_binary, y_score)

        if method == "SWE":
            print("best_t, best_l:        \n", best_T, best_l)
        elif method != "dmp_inf":
            print("best_t:        \n", best_T)
        print("RMSE:        \n", rmse)
        print("MAE:         \n", mae)
        print("Pearson:     \n", pearson.squeeze().numpy())
        print("Spearman:    \n", spearman)
        print("AUC:         \n", auc)
        print("Time Approx: \n", time_ic_approx)

-------------SWE-------------
best_t, best_l:        
 7 2
RMSE:        
 0.009054967236836518
MAE:         
 tensor(0.0058)
Pearson:     
 0.99853605
Spearman:    
 0.9933835887584672
AUC:         
 0.999870386950763
Time Approx: 
 0.06110072135925293
-------------um_IC-------------
best_t:        
 8
RMSE:        
 0.008981242748214765
MAE:         
 tensor(0.0058)
Pearson:     
 0.9985042
Spearman:    
 0.993362763510442
AUC:         
 0.9998713061922471
Time Approx: 
 0.06270813941955566
-------------dmp_um_fast-------------
best_t:        
 8
RMSE:        
 0.0696328548579781
MAE:         
 tensor(0.0141)
Pearson:     
 0.90004575
Spearman:    
 0.9622994077951386
AUC:         
 0.951197075656463
Time Approx: 
 0.19265985488891602
-------------Naive-------------
best_t:        
 8
RMSE:        
 0.06970903259059348
MAE:         
 tensor(0.0143)
Pearson:     
 0.9001261
Spearman:    
 0.961952980567893
AUC:         
 0.951235946439213
Time Approx: 
 0.11375999450683594
------------

### Extensive Experiments

In [10]:
num_sim=10000
percentile=95

num_graphs = 10

G_anybeat = load_directed_graph_from_edges("soc-anybeat/soc-anybeat.edges")

methods = [
    "SWE",          #0
    "um_IC",        #1
    "dmp_um_fast",  #2
    "Naive",        #3
    "dmp_ic_fast",  #4
    "dmp_inf",      #5
    "modified_ALE", #6
    "ALE_heuristic" #7
]

best_par = {}
for method in methods:
    best_par[method] = (0,0)

dataset="Anybeat"

for prob_selected in [0.01, 0.1, 0.2]:
    #for high in [0.1, 0.5, 1]:
    for i in range(num_graphs+1):
        edge_dict_anybeat = generate_edge_probabilities(G_anybeat,method="degree_based",low=0, high=high)
        mask = torch.rand(len(G_anybeat.nodes())) < prob_selected
        prior_probs_anybeat = mask.float()  * torch.rand(len(G_anybeat.nodes()))

        G = G_anybeat
        prior_probs = prior_probs_anybeat
        edge_dict = edge_dict_anybeat
        
        start = time.time()
        ic = optimized_independent_cascade(G, prior_probs, edge_dict, num_sim)
        end = time.time()
        time_mc = end - start
        true_posterior_tensor = torch.tensor(
            [ic[node] for node in ic],
            dtype=torch.float
        ).view(-1, 1)
        true_probs = true_posterior_tensor.squeeze().numpy()

        for approximation in methods:
            if approximation == "SWE":
                if i==0:
                    best_T, best_l = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, best_l)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = SWE(G, edge_dict, prior_probs, T=best_T, a=1, layers=best_l)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "um_IC":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = um_IC(G, edge_dict, prior_probs, best_T)[best_T]
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "dmp_um_fast":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = dmp_um_fast(G, prior_probs, edge_dict, best_T, eps=1e-20)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "Naive":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = Naive(G, edge_dict, prior_probs, best_T, eps=1e-12)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "dmp_ic_fast":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "dmp_inf":
                if i==0:
                    best_par[approximation] = (0, 0)
                    continue
                start = time.time()
                preds = dmp_inf(G, prior_probs, edge_dict, eps=1e-20, max_iter=100)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "modified_ALE":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = modified_ALE(G, prior_probs, edge_dict, best_T)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "ALE_heuristic":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = ALE_heuristic(G, prior_probs, edge_dict, best_T)
                end = time.time()
                time_ic_apporx = end - start
            
            preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
            rmse = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5
        
            pearson = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
            spearman, _ = spearmanr(preds_IC_approx.squeeze(),
                                      true_posterior_tensor.squeeze())
        
            y_true = true_posterior_tensor.squeeze().numpy()
            y_score = preds_IC_approx.squeeze().numpy()
            threshold = np.percentile(y_true, percentile)
            y_binary = (y_true >= threshold).astype(int)
            auc = roc_auc_score(y_binary, y_score)

            edge_method = "degree_based"#"random_" + str(high)

            append_to_dataset(
                file_path="IC_APPROX_DATASET.csv",
                dataset=dataset,
                prob_selected=prob_selected,
                edge_method=edge_method,
                time_mc=time_mc,
                approximation=approximation,
                T=best_T,
                L=best_l,
                time_ic_apporx=time_ic_apporx,
                rmse=rmse,
                pearson=pearson,
                spearman=spearman,
                auc=auc
            )
            
        print("dataset: ", dataset, 
              ", prob_selected: ", prob_selected,
              #", high: ", high, 
              ", graph: ", i)

/tmp/ipykernel_3017603/2599858617.py:50: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
/tmp/ipykernel_3017603/2599858617.py:62: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
/tmp/ipykernel_3017603/2599858617.py:74: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so 

dataset:  Anybeat , prob_selected:  0.01 , graph:  0
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv


/tmp/ipykernel_3017603/2599858617.py:104: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
dataset:  Anybeat , prob_selected:  0.01 , graph:  1
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv


/tmp/ipykernel_3017603/2599858617.py:104: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


...

Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
dataset:  Anybeat , prob_selected:  0.2 , graph:  9


/tmp/ipykernel_3017603/2599858617.py:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)


Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv


/tmp/ipykernel_3017603/2599858617.py:104: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
dataset:  Anybeat , prob_selected:  0.2 , graph:  10


## Advogato

### For Debugging

In [16]:
prob_selected = 0.01
method="degree_based"
num_sim=1000
low=0
high=0.1
percentile=95
T=5

G_advogato, weights_advogato = load_weighted_directed_graph("soc-advogato/soc-advogato.edges")
edge_dict_advogato = generate_edge_probabilities(G_advogato,method=method,low=low, high=high)

for edge in edge_dict_advogato:
    edge_dict_advogato[edge] *= weights_advogato[edge]
    
mask = torch.rand(len(G_advogato.nodes())) < prob_selected
prior_probs_advogato = mask.float()  * torch.rand(len(G_advogato.nodes()))

start = time.time()
ic_advogato = optimized_independent_cascade(G_advogato, prior_probs_advogato, edge_dict_advogato, num_sim)
end = time.time()
time_mc_advogato = end - start
true_posterior_tensor_advogato = torch.tensor(
    [ic_advogato[node] for node in ic_advogato],
    dtype=torch.float
).view(-1, 1)
true_probs = true_posterior_tensor_advogato.squeeze().numpy()
print("Time for MC: ", time_mc_advogato, " seconds")

Time for MC:  1.024778127670288  seconds


In [17]:
G=G_advogato
edge_dict = edge_dict_advogato
prior_probs = prior_probs_advogato
methods = [
    "SWE",          #0
    "um_IC",        #1
    "dmp_um_fast",  #2
    "Naive",        #3
    "dmp_ic_fast",  #4
    "dmp_inf",      #5
    "modified_ALE", #6
    "ALE_heuristic" #7
]
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for method in methods:
        print(f"-------------{method}-------------")
        if method == "SWE":
            best_T, best_l = get_best_parameters(
            G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
            )
            start = time.time()
            preds = SWE(G, edge_dict, np.array(prior_probs), best_T, a=1, layers=best_l)
        elif method == "um_IC":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "um_IC", max_t=10)
            start = time.time()
            preds = um_IC(G, edge_dict, prior_probs, best_T)[best_T]
        elif method == "dmp_um_fast":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "dmp_um_fast")
            start = time.time()
            preds = dmp_um_fast(G, prior_probs, edge_dict, best_T, eps=1e-20)
        elif method == "Naive":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "Naive", max_t=10)
            start = time.time()
            preds = Naive(G, edge_dict, prior_probs, best_T, eps=1e-12)
        elif method == "dmp_ic_fast":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "dmp_ic_fast")
            start = time.time()
            preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)
        elif method == "dmp_inf":
            start = time.time()
            preds = dmp_inf(G, prior_probs, edge_dict, eps=1e-20, max_iter=100)
        elif method == "modified_ALE":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "modified_ALE")
            start = time.time()
            preds = modified_ALE(G, prior_probs, edge_dict, best_T)
        elif method == "ALE_heuristic":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "ALE_heuristic")
            start = time.time()
            preds = ALE_heuristic(G, prior_probs, edge_dict, best_T)
        end = time.time()
        time_ic_approx = end - start

        true_posterior_tensor = true_posterior_tensor_advogato
        preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
        rmse = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5

        mae = abs(true_posterior_tensor.squeeze()-preds_IC_approx.squeeze()).mean()
        
        pearson = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
        spearman, _ = spearmanr(preds_IC_approx.squeeze(),
                                  true_posterior_tensor.squeeze())
        
        y_true = true_posterior_tensor.squeeze().numpy()
        y_score = preds_IC_approx.squeeze().numpy()
        threshold = np.percentile(y_true, percentile)
        y_binary = (y_true >= threshold).astype(int)
        auc = roc_auc_score(y_binary, y_score)

        if method == "SWE":
            print("best_t, best_l:        \n", best_T, best_l)
        elif method != "dmp_inf":
            print("best_t:        \n", best_T)
        print("RMSE:        \n", rmse)
        print("MAE:         \n", mae)
        print("Pearson:     \n", pearson.squeeze().numpy())
        print("Spearman:    \n", spearman)
        print("AUC:         \n", auc)
        print("Time Approx: \n", time_ic_approx)

-------------SWE-------------
best_t, best_l:        
 5 0
RMSE:        
 0.0076747990417647795
MAE:         
 tensor(0.0028)
Pearson:     
 0.99359554
Spearman:    
 0.9168396615229726
AUC:         
 0.9980430477967487
Time Approx: 
 0.05904555320739746
-------------um_IC-------------
best_t:        
 4
RMSE:        
 0.011634021793729962
MAE:         
 tensor(0.0037)
Pearson:     
 0.9885743
Spearman:    
 0.9126669308836349
AUC:         
 0.9979919989059614
Time Approx: 
 0.05621695518493652
-------------dmp_um_fast-------------
best_t:        
 7
RMSE:        
 0.008381511038402567
MAE:         
 tensor(0.0025)
Pearson:     
 0.9937668
Spearman:    
 0.9236341108668724
AUC:         
 0.9981044001333829
Time Approx: 
 0.1998276710510254
-------------Naive-------------
best_t:        
 2
RMSE:        
 0.012555705613651076
MAE:         
 tensor(0.0060)
Pearson:     
 0.98743534
Spearman:    
 0.8276602707750076
AUC:         
 0.9950447450552826
Time Approx: 
 0.08301353454589844
----

### Extensive Experiments

In [8]:
num_sim=10000
percentile=95

num_graphs = 10

G_advogato, weights_advogato = load_weighted_directed_graph("soc-advogato/soc-advogato.edges")
#edge_dict_advogato = generate_edge_probabilities(G_advogato,method=method,low=low, high=high)

#for edge in edge_dict_advogato:
#    edge_dict_advogato[edge] *= weights_advogato[edge]

methods = [
    "SWE",          #0
    "um_IC",        #1
    "dmp_um_fast",  #2
    "Naive",        #3
    "dmp_ic_fast",  #4
    "dmp_inf",      #5
    "modified_ALE", #6
    "ALE_heuristic" #7
]

best_par = {}
for method in methods:
    best_par[method] = (0,0)

dataset="Advogato"

for prob_selected in [0.01, 0.1, 0.2]:
    #for high in [0.1, 0.5, 1]:
    for i in range(num_graphs+1):
        edge_dict_advogato = generate_edge_probabilities(G_advogato,method="degree_based",low=0, high=high)
        mask = torch.rand(len(G_advogato.nodes())) < prob_selected
        prior_probs_advogato = mask.float()  * torch.rand(len(G_advogato.nodes()))

        G = G_advogato
        prior_probs = prior_probs_advogato
        edge_dict = edge_dict_advogato
        
        start = time.time()
        ic = optimized_independent_cascade(G, prior_probs, edge_dict, num_sim)
        end = time.time()
        time_mc = end - start
        true_posterior_tensor = torch.tensor(
            [ic[node] for node in ic],
            dtype=torch.float
        ).view(-1, 1)
        true_probs = true_posterior_tensor.squeeze().numpy()

        for approximation in methods:
            if approximation == "SWE":
                if i==0:
                    best_T, best_l = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, best_l)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = SWE(G, edge_dict, prior_probs, T=best_T, a=1, layers=best_l)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "um_IC":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = um_IC(G, edge_dict, prior_probs, best_T)[best_T]
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "dmp_um_fast":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = dmp_um_fast(G, prior_probs, edge_dict, best_T, eps=1e-20)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "Naive":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = Naive(G, edge_dict, prior_probs, best_T, eps=1e-12)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "dmp_ic_fast":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "dmp_inf":
                if i==0:
                    best_par[approximation] = (0, 0)
                    continue
                start = time.time()
                preds = dmp_inf(G, prior_probs, edge_dict, eps=1e-20, max_iter=100)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "modified_ALE":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = modified_ALE(G, prior_probs, edge_dict, best_T)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "ALE_heuristic":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = ALE_heuristic(G, prior_probs, edge_dict, best_T)
                end = time.time()
                time_ic_apporx = end - start
            
            preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
            rmse = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5
        
            pearson = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
            spearman, _ = spearmanr(preds_IC_approx.squeeze(),
                                      true_posterior_tensor.squeeze())
        
            y_true = true_posterior_tensor.squeeze().numpy()
            y_score = preds_IC_approx.squeeze().numpy()
            threshold = np.percentile(y_true, percentile)
            y_binary = (y_true >= threshold).astype(int)
            auc = roc_auc_score(y_binary, y_score)

            edge_method = "degree_based"#"random_" + str(high)

            append_to_dataset(
                file_path="IC_APPROX_DATASET.csv",
                dataset=dataset,
                prob_selected=prob_selected,
                edge_method=edge_method,
                time_mc=time_mc,
                approximation=approximation,
                T=best_T,
                L=best_l,
                time_ic_apporx=time_ic_apporx,
                rmse=rmse,
                pearson=pearson,
                spearman=spearman,
                auc=auc
            )
            
        print("dataset: ", dataset, 
              ", prob_selected: ", prob_selected,
              #", high: ", high, 
              ", graph: ", i)

/tmp/ipykernel_3017603/2560486532.py:54: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
/tmp/ipykernel_3017603/2560486532.py:66: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
/tmp/ipykernel_3017603/2560486532.py:78: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so 

dataset:  Advogato , prob_selected:  0.01 , graph:  0
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv


/tmp/ipykernel_3017603/2560486532.py:108: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
dataset:  Advogato , prob_selected:  0.01 , graph:  1
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv


/tmp/ipykernel_3017603/2560486532.py:108: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


...

Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
dataset:  Advogato , prob_selected:  0.2 , graph:  8
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv


/tmp/ipykernel_3017603/2560486532.py:108: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
dataset:  Advogato , prob_selected:  0.2 , graph:  9
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv


/tmp/ipykernel_3017603/2560486532.py:108: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
dataset:  Advogato , prob_selected:  0.2 , graph:  10


## Gemsec

### For Debugging

In [18]:
prob_selected = 0.01
method="random"
num_sim=1000
low=0
high=0.1
percentile=95
T=5

G_gemsec = load_directed_graph_from_edges_cs("soc-gemsec-HR/soc-gemsec-HR.edges")
edge_dict_gemsec = generate_edge_probabilities(G_gemsec,method=method,low=low, high=high)
    
mask = torch.rand(len(G_gemsec.nodes())) < prob_selected
prior_probs_gemsec = mask.float()  * torch.rand(len(G_gemsec.nodes()))

start = time.time()
ic_gemsec = optimized_independent_cascade(G_gemsec, prior_probs_gemsec, edge_dict_gemsec, num_sim)
end = time.time()
time_mc_gemsec = end - start
true_posterior_tensor_gemsec = torch.tensor(
    [ic_gemsec[node] for node in ic_gemsec],
    dtype=torch.float
).view(-1, 1)
true_probs = true_posterior_tensor_gemsec.squeeze().numpy()
print("Time for MC: ", time_mc_gemsec, " seconds")

Time for MC:  6.071696043014526  seconds


In [19]:
G=G_gemsec
edge_dict = edge_dict_gemsec
prior_probs = prior_probs_gemsec
methods = [
    "SWE",          #0
    "um_IC",        #1
    "dmp_um_fast",  #2
    "Naive",        #3
    "dmp_ic_fast",  #4
    "dmp_inf",      #5
    "modified_ALE", #6
    "ALE_heuristic" #7
]
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for method in methods:
        print(f"-------------{method}-------------")
        if method == "SWE":
            best_T, best_l = get_best_parameters(
            G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=5, max_layers=5
            )
            start = time.time()
            preds = SWE(G, edge_dict, np.array(prior_probs), best_T, a=1, layers=best_l)
        elif method == "um_IC":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "um_IC", max_t=10)
            start = time.time()
            preds = um_IC(G, edge_dict, prior_probs, best_T)[best_T]
        elif method == "dmp_um_fast":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "dmp_um_fast")
            start = time.time()
            preds = dmp_um_fast(G, prior_probs, edge_dict, best_T, eps=1e-20)
        elif method == "Naive":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "Naive", max_t=10)
            start = time.time()
            preds = Naive(G, edge_dict, prior_probs, best_T, eps=1e-12)
        elif method == "dmp_ic_fast":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "dmp_ic_fast")
            start = time.time()
            preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)
        elif method == "dmp_inf":
            start = time.time()
            preds = dmp_inf(G, prior_probs, edge_dict, eps=1e-20, max_iter=100)
        elif method == "modified_ALE":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "modified_ALE")
            start = time.time()
            preds = modified_ALE(G, prior_probs, edge_dict, best_T)
        elif method == "ALE_heuristic":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "ALE_heuristic")
            start = time.time()
            preds = ALE_heuristic(G, prior_probs, edge_dict, best_T)
        end = time.time()
        time_ic_approx = end - start

        true_posterior_tensor = true_posterior_tensor_gemsec
        preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
        rmse = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5

        mae = abs(true_posterior_tensor.squeeze()-preds_IC_approx.squeeze()).mean()
        
        pearson = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
        spearman, _ = spearmanr(preds_IC_approx.squeeze(),
                                  true_posterior_tensor.squeeze())
        
        y_true = true_posterior_tensor.squeeze().numpy()
        y_score = preds_IC_approx.squeeze().numpy()
        threshold = np.percentile(y_true, percentile)
        y_binary = (y_true >= threshold).astype(int)
        auc = roc_auc_score(y_binary, y_score)

        if method == "SWE":
            print("best_t, best_l:        \n", best_T, best_l)
        elif method != "dmp_inf":
            print("best_t:        \n", best_T)
        print("RMSE:        \n", rmse)
        print("MAE:         \n", mae)
        print("Pearson:     \n", pearson.squeeze().numpy())
        print("Spearman:    \n", spearman)
        print("AUC:         \n", auc)
        print("Time Approx: \n", time_ic_approx)

-------------SWE-------------
best_t, best_l:        
 3 3
RMSE:        
 0.0024701039287743183
MAE:         
 tensor(0.0010)
Pearson:     
 0.9990868
Spearman:    
 0.8571082001197488
AUC:         
 0.999321590874049
Time Approx: 
 0.4688858985900879
-------------um_IC-------------
best_t:        
 6
RMSE:        
 0.0024701529910684367
MAE:         
 tensor(0.0010)
Pearson:     
 0.9990869
Spearman:    
 0.8571081391016802
AUC:         
 0.9993214993075531
Time Approx: 
 0.4726839065551758
-------------dmp_um_fast-------------
best_t:        
 10
RMSE:        
 0.08093793342356453
MAE:         
 tensor(0.0151)
Pearson:     
 0.018550502
Spearman:    
 0.6103089818851196
AUC:         
 0.7361967803726737
Time Approx: 
 1.3207964897155762
-------------Naive-------------
best_t:        
 10
RMSE:        
 0.08093795068350254
MAE:         
 tensor(0.0151)
Pearson:     
 0.018554576
Spearman:    
 0.6103022282892253
AUC:         
 0.7361493630148526
Time Approx: 
 0.7479972839355469
-----

### Extensive Experiments

In [12]:
num_sim=10000
percentile=95

num_graphs = 10

G_gemsec = load_directed_graph_from_edges_cs("soc-gemsec-HR/soc-gemsec-HR.edges")

methods = [
    "SWE",          #0
    "um_IC",        #1
    "dmp_um_fast",  #2
    "Naive",        #3
    "dmp_ic_fast",  #4
    "dmp_inf",      #5
    "modified_ALE", #6
    "ALE_heuristic" #7
]

best_par = {}
for method in methods:
    best_par[method] = (0,0)

dataset="Gemsec"

for prob_selected in [0.01, 0.1, 0.2]:
    #for high in [0.1, 0.5, 1]:
    for i in range(num_graphs+1):
        edge_dict_gemsec = generate_edge_probabilities(G_gemsec,method="degree_based",low=0, high=high)
        mask = torch.rand(len(G_gemsec.nodes())) < prob_selected
        prior_probs_gemsec = mask.float()  * torch.rand(len(G_gemsec.nodes()))

        G = G_gemsec
        prior_probs = prior_probs_gemsec
        edge_dict = edge_dict_gemsec
        
        start = time.time()
        ic = optimized_independent_cascade(G, prior_probs, edge_dict, num_sim)
        end = time.time()
        time_mc = end - start
        true_posterior_tensor = torch.tensor(
            [ic[node] for node in ic],
            dtype=torch.float
        ).view(-1, 1)
        true_probs = true_posterior_tensor.squeeze().numpy()

        for approximation in methods:
            if approximation == "SWE":
                if i==0:
                    best_T, best_l = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, best_l)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = SWE(G, edge_dict, prior_probs, T=best_T, a=1, layers=best_l)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "um_IC":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = um_IC(G, edge_dict, prior_probs, best_T)[best_T]
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "dmp_um_fast":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = dmp_um_fast(G, prior_probs, edge_dict, best_T, eps=1e-20)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "Naive":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = Naive(G, edge_dict, prior_probs, best_T, eps=1e-12)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "dmp_ic_fast":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "dmp_inf":
                if i==0:
                    best_par[approximation] = (0, 0)
                    continue
                start = time.time()
                preds = dmp_inf(G, prior_probs, edge_dict, eps=1e-20, max_iter=100)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "modified_ALE":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = modified_ALE(G, prior_probs, edge_dict, best_T)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "ALE_heuristic":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = ALE_heuristic(G, prior_probs, edge_dict, best_T)
                end = time.time()
                time_ic_apporx = end - start
            
            preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
            rmse = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5
        
            pearson = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
            spearman, _ = spearmanr(preds_IC_approx.squeeze(),
                                      true_posterior_tensor.squeeze())
        
            y_true = true_posterior_tensor.squeeze().numpy()
            y_score = preds_IC_approx.squeeze().numpy()
            threshold = np.percentile(y_true, percentile)
            y_binary = (y_true >= threshold).astype(int)
            auc = roc_auc_score(y_binary, y_score)

            edge_method = "degree_based"#"random_" + str(high)

            append_to_dataset(
                file_path="IC_APPROX_DATASET.csv",
                dataset=dataset,
                prob_selected=prob_selected,
                edge_method=edge_method,
                time_mc=time_mc,
                approximation=approximation,
                T=best_T,
                L=best_l,
                time_ic_apporx=time_ic_apporx,
                rmse=rmse,
                pearson=pearson,
                spearman=spearman,
                auc=auc
            )
            
        print("dataset: ", dataset, 
              ", prob_selected: ", prob_selected,
              #", high: ", high, 
              ", graph: ", i)

/tmp/ipykernel_3017603/1065171256.py:50: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
/tmp/ipykernel_3017603/1065171256.py:62: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
/tmp/ipykernel_3017603/1065171256.py:74: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so 

dataset:  Gemsec , prob_selected:  0.01 , graph:  0


/tmp/ipykernel_3017603/1065171256.py:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)


Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv


/tmp/ipykernel_3017603/1065171256.py:104: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


...

Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
dataset:  Gemsec , prob_selected:  0.2 , graph:  9


/tmp/ipykernel_3017603/1065171256.py:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)


Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv


/tmp/ipykernel_3017603/1065171256.py:104: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
dataset:  Gemsec , prob_selected:  0.2 , graph:  10


## Epinions

### For Debugging

In [89]:
prob_selected = 0.01
method="random"
num_sim=1000
low=0
high=1
percentile=95
T=5
G_epinions = load_directed_graph_from_mtx("soc-sign-epinions/soc-sign-epinions.mtx")
edge_dict_epinions = generate_edge_probabilities(G_epinions,method=method,low=low, high=high)
    
mask = torch.rand(len(G_epinions.nodes())) < prob_selected
prior_probs_epinions = mask.float()  * torch.rand(len(G_epinions.nodes()))

start = time.time()
ic_epinions = optimized_independent_cascade(G_epinions, prior_probs_epinions, edge_dict_epinions, num_sim)
end = time.time()
time_mc_epinions = end - start
true_posterior_tensor_epinions = torch.tensor(
    [ic_epinions[node] for node in ic_epinions],
    dtype=torch.float
).view(-1, 1)
true_probs = true_posterior_tensor_epinions.squeeze().numpy()
print("Time for MC: ", time_mc_epinions, " seconds")

Time for MC:  338.93852615356445  seconds


In [92]:
G=G_epinions
edge_dict = edge_dict_epinions
prior_probs = prior_probs_epinions
methods = [
    "SWE",          #0
    "um_IC",        #1
    "dmp_um_fast",  #2
    "Naive",        #3
    "dmp_ic_fast",  #4
    "dmp_inf",      #5
    "modified_ALE", #6
    "ALE_heuristic" #7
]
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for method in methods:
        print(f"-------------{method}-------------")
        if method == "SWE":
            best_T, best_l = get_best_parameters(
            G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
            )
            start = time.time()
            preds = SWE(G, edge_dict, np.array(prior_probs), best_T, a=1, layers=best_l)
        elif method == "um_IC":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "um_IC", max_t=10)
            start = time.time()
            preds = um_IC(G, edge_dict, prior_probs, best_T)[best_T]
        elif method == "dmp_um_fast":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "dmp_um_fast")
            start = time.time()
            preds = dmp_um_fast(G, prior_probs, edge_dict, best_T, eps=1e-20)
        elif method == "Naive":
            best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "Naive", max_t=10)
            start = time.time()
            preds = Naive(G, edge_dict, prior_probs, best_T, eps=1e-12)
        elif method == "dmp_ic_fast":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "dmp_ic_fast")
            start = time.time()
            preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)
        elif method == "dmp_inf":
            start = time.time()
            preds = dmp_inf(G, prior_probs, edge_dict, eps=1e-20, max_iter=100)
        elif method == "modified_ALE":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "modified_ALE")
            start = time.time()
            preds = modified_ALE(G, prior_probs, edge_dict, best_T)
        elif method == "ALE_heuristic":
            best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "ALE_heuristic")
            start = time.time()
            preds = ALE_heuristic(G, prior_probs, edge_dict, best_T)
        end = time.time()
        time_ic_approx = end - start

        true_posterior_tensor = true_posterior_tensor_epinions
        preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
        rmse = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5
        
        pearson = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
        spearman, _ = spearmanr(preds_IC_approx.squeeze(),
                                  true_posterior_tensor.squeeze())
        
        y_true = true_posterior_tensor.squeeze().numpy()
        y_score = preds_IC_approx.squeeze().numpy()
        threshold = np.percentile(y_true, percentile)
        y_binary = (y_true >= threshold).astype(int)
        auc = roc_auc_score(y_binary, y_score)

        if method == "SWE":
            print("best_t, best_l:        \n", best_T, best_l)
        elif method != "dmp_inf":
            print("best_t:        \n", best_T)
        print("RMSE:        \n", rmse)
        print("Pearson:     \n", pearson.squeeze().numpy())
        print("Spearman:    \n", spearman)
        print("AUC:         \n", auc)
        print("Time Approx: \n", time_ic_approx)

-------------SWE-------------
best_t, best_l:        
 3 3
RMSE:        
 0.015306897617992227
Pearson:     
 0.9993395
Spearman:    
 0.999035454151648
AUC:         
 0.9991394227213766
Time Approx: 
 0.8375604152679443
-------------um_IC-------------
best_t:        
 7
RMSE:        
 0.04335953324735821
Pearson:     
 0.9957595
Spearman:    
 0.99732074147928
AUC:         
 0.99775519459562
Time Approx: 
 0.7696647644042969
-------------dmp_um_fast-------------
best_t:        
 6
RMSE:        
 0.06376757304761659
Pearson:     
 0.98833686
Spearman:    
 0.9870661724216075
AUC:         
 0.9990728158025126
Time Approx: 
 3.0186495780944824
-------------Naive-------------
best_t:        
 6
RMSE:        
 0.064482277721687
Pearson:     
 0.988143
Spearman:    
 0.9870200599295682
AUC:         
 0.9990150026309085
Time Approx: 
 1.4456820487976074
-------------dmp_ic_fast-------------
best_t:        
 5
RMSE:        
 0.06298611881181461
Pearson:     
 0.98864007
Spearman:    
 0.98721

In [93]:
print(f"-------------{method}-------------")
method="SWE"
if method == "SWE":
    #best_T, best_l = get_best_parameters(
    #G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
    #)
    start = time.time()
    preds = SWE(G, edge_dict, np.array(prior_probs), 4, a=1, layers=2)
elif method == "um_IC":
    best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "um_IC", max_t=10)
    start = time.time()
    preds = um_IC(G, edge_dict, prior_probs, best_T)[best_T]
elif method == "dmp_um_fast":
    best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "dmp_um_fast")
    start = time.time()
    preds = dmp_um_fast(G, prior_probs, edge_dict, best_T, eps=1e-20)
elif method == "Naive":
    best_T = get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "Naive", max_t=10)
    start = time.time()
    preds = Naive(G, edge_dict, prior_probs, best_T, eps=1e-12)
elif method == "dmp_ic_fast":
    best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "dmp_ic_fast")
    start = time.time()
    preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)
elif method == "dmp_inf":
    start = time.time()
    preds = dmp_inf(G, prior_probs, edge_dict, eps=1e-20, max_iter=100)
elif method == "modified_ALE":
    best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "modified_ALE")
    start = time.time()
    preds = modified_ALE(G, prior_probs, edge_dict, best_T)
elif method == "ALE_heuristic":
    best_T= get_best_parameters(G, edge_dict, np.array(prior_probs), true_probs, method = "ALE_heuristic")
    start = time.time()
    preds = ALE_heuristic(G, prior_probs, edge_dict, best_T)
end = time.time()
time_ic_approx = end - start

true_posterior_tensor = true_posterior_tensor_epinions
preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
rmse = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5

pearson = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
spearman, _ = spearmanr(preds_IC_approx.squeeze(),
                          true_posterior_tensor.squeeze())

y_true = true_posterior_tensor.squeeze().numpy()
y_score = preds_IC_approx.squeeze().numpy()
threshold = np.percentile(y_true, percentile)
y_binary = (y_true >= threshold).astype(int)
auc = roc_auc_score(y_binary, y_score)

if method == "SWE":
    print("best_t, best_l:        \n", best_T, best_l)
elif method != "dmp_inf":
    print("best_t:        \n", best_T)
print("RMSE:        \n", rmse)
print("Pearson:     \n", pearson.squeeze().numpy())
print("Spearman:    \n", spearman)
print("AUC:         \n", auc)
print("Time Approx: \n", time_ic_approx)

-------------ALE_heuristic-------------


/tmp/ipykernel_2794150/2141434577.py:8: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = SWE(G, edge_dict, np.array(prior_probs), 4, a=1, layers=2)


best_t, best_l:        
 3 3
RMSE:        
 0.016403972696163935
Pearson:     
 0.9992292
Spearman:    
 0.9990089694454375
AUC:         
 0.9991572938642969
Time Approx: 
 1.062340259552002


### Extensive experiments

In [6]:
num_sim=10000
percentile=95

num_graphs = 10

G_epinions = load_directed_graph_from_mtx("soc-sign-epinions/soc-sign-epinions.mtx")

methods = [
    "SWE",          #0
    "um_IC",        #1
    "dmp_um_fast",  #2
    "Naive",        #3
    "dmp_ic_fast",  #4
    "dmp_inf",      #5
    "modified_ALE", #6
    "ALE_heuristic" #7
]

best_par = {}
for method in methods:
    best_par[method] = (0,0)

dataset="Epinions"

for prob_selected in [0.01, 0.1, 0.2]:
    #for high in [0.1, 0.5, 1]:
    for i in range(num_graphs+1):
        edge_dict_epinions = generate_edge_probabilities(G_epinions,method="degree_based",low=0, high=high)
        mask = torch.rand(len(G_epinions.nodes())) < prob_selected
        prior_probs_epinions = mask.float()  * torch.rand(len(G_epinions.nodes()))

        G = G_epinions
        prior_probs = prior_probs_epinions
        edge_dict = edge_dict_epinions
        
        start = time.time()
        ic = optimized_independent_cascade(G, prior_probs, edge_dict, num_sim)
        end = time.time()
        time_mc = end - start
        true_posterior_tensor = torch.tensor(
            [ic[node] for node in ic],
            dtype=torch.float
        ).view(-1, 1)
        true_probs = true_posterior_tensor.squeeze().numpy()

        for approximation in methods:
            if approximation == "SWE":
                if i==0:
                    best_T, best_l = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, best_l)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = SWE(G, edge_dict, prior_probs, T=best_T, a=1, layers=best_l)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "um_IC":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = um_IC(G, edge_dict, prior_probs, best_T)[best_T]
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "dmp_um_fast":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = dmp_um_fast(G, prior_probs, edge_dict, best_T, eps=1e-20)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "Naive":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = Naive(G, edge_dict, prior_probs, best_T, eps=1e-12)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "dmp_ic_fast":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "dmp_inf":
                if i==0:
                    best_par[approximation] = (0, 0)
                    continue
                start = time.time()
                preds = dmp_inf(G, prior_probs, edge_dict, eps=1e-20, max_iter=100)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "modified_ALE":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = modified_ALE(G, prior_probs, edge_dict, best_T)
                end = time.time()
                time_ic_apporx = end - start
            elif approximation == "ALE_heuristic":
                if i==0:
                    best_T = get_best_parameters(
                    G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
                    )
                    best_par[approximation] = (best_T, 0)
                    continue
                best_T, best_l = best_par[approximation]
                start = time.time()
                preds = ALE_heuristic(G, prior_probs, edge_dict, best_T)
                end = time.time()
                time_ic_apporx = end - start
            
            preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
            rmse = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5
        
            pearson = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
            spearman, _ = spearmanr(preds_IC_approx.squeeze(),
                                      true_posterior_tensor.squeeze())
        
            y_true = true_posterior_tensor.squeeze().numpy()
            y_score = preds_IC_approx.squeeze().numpy()
            threshold = np.percentile(y_true, percentile)
            y_binary = (y_true >= threshold).astype(int)
            auc = roc_auc_score(y_binary, y_score)

            edge_method = "degree_based"#"random_" + str(high)

            append_to_dataset(
                file_path="IC_APPROX_DATASET.csv",
                dataset=dataset,
                prob_selected=prob_selected,
                edge_method=edge_method,
                time_mc=time_mc,
                approximation=approximation,
                T=best_T,
                L=best_l,
                time_ic_apporx=time_ic_apporx,
                rmse=rmse,
                pearson=pearson,
                spearman=spearman,
                auc=auc
            )
            
        print("dataset: ", dataset, 
              ", prob_selected: ", prob_selected,
              #", high: ", high, 
              ", graph: ", i)

/tmp/ipykernel_3455152/2059785056.py:50: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  G, edge_dict, np.array(prior_probs), true_probs, method = "SWE", max_t=10, max_layers=5
/tmp/ipykernel_3455152/2059785056.py:62: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  G, edge_dict, np.array(prior_probs), true_probs, method = approximation, max_t=10, max_layers=5
/tmp/ipykernel_3455152/2059785056.py:74: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so 

dataset:  Epinions , prob_selected:  0.01 , graph:  0


/tmp/ipykernel_3455152/2059785056.py:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)


Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv


/tmp/ipykernel_3455152/2059785056.py:104: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


...

Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
dataset:  Epinions , prob_selected:  0.2 , graph:  9


/tmp/ipykernel_3455152/2059785056.py:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)


Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv


/tmp/ipykernel_3455152/2059785056.py:104: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  preds = dmp_ic_fast(G, np.array(prior_probs), edge_dict, best_T)


Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
Appended new row to IC_APPROX_DATASET.csv
dataset:  Epinions , prob_selected:  0.2 , graph:  10
